In [30]:
import bqplot
import ytree
import yt
from collections import defaultdict
import ipywidgets as widgets
from traitlets import Unicode, validate, default, List, TraitError
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw)

In [3]:
a = ytree.load("rockstar_halos/out_0.list")

Performance will be improved by saving this arbor with "save_arbor" and reloading:
	>>> a = ytree.load("rockstar_halos/out_0.list")
	>>> fn = a.save_arbor()
	>>> a = ytree.load(fn)


In [4]:
print(a.derived_field_list)
print(a.field_list)

['halo_id', 'desc_id', 'mass', 'virial_mass', 'virial_radius', 'scale_radius', 'velocity_dispersion', 'position_x', 'position_y', 'position_z', 'velocity_x', 'velocity_y', 'velocity_z', 'angular_momentum_x', 'angular_momentum_y', 'angular_momentum_z', 'spin_parameter', 'T_|U|', 'redshift']
['ID', 'DescID', 'Mvir', 'Vmax', 'Vrms', 'Rvir', 'Rs', 'Np', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'JX', 'JY', 'JZ', 'Spin', 'rs_klypin', 'Mvir_all', 'M200b', 'M200c', 'M500c', 'M2500c', 'Xoff', 'Voff', 'spin_bullock', 'b_to_a', 'c_to_a', 'A[x]', 'A[y]', 'A[z]', 'b_to_a(500c)', 'c_to_a(500c)', 'A[x](500c)', 'A[y](500c)', 'A[z](500c)', 'T/|U|', 'M_pe_Behroozi', 'M_pe_Diemer', 'scale_factor', 'uid', 'desc_uid']


In [5]:
# using pandas to make redshift dict

# df_trees = {}
# counter = 0
# for tree in a:
#     scale = tree["prog", "scale_factor"]
#     assert(scale.size == np.unique(scale).size)
#     Mvir = tree["prog", "Mvir_all"]
#     Vrms = tree["prog", "Vrms"]
#     Rvir = tree["prog", "Rvir"]
#     Spin = tree["prog", "Spin"]
#     Z = tree["prog", "redshift"]
#     ti = str(counter)
#     d = pd.DataFrame({('Mvir'): Mvir, ('Vrms'): Vrms, ('Rvir'): Rvir,('Spin'): Spin,('redshift'): Z})
#     df_trees[ti] = d
#     counter += 1

# df = pd.concat(df_trees)
# df2 = df.unstack(0)

# redshift_dict = {}
# for row in df2["redshift"].itertuples():
#     idx = row[0]
#     key = np.round(np.max(row[1]),2)
#     redshift_dict[key] = {'Rvir':np.array(df2["Rvir"].loc[idx]), 'Mvir':np.array(df2["Mvir"].loc[idx]),
#                           'Spin':np.array(df2["Spin"].loc[idx]),'Vrms':np.array(df2["Vrms"].loc[idx])}

In [124]:
# LINKED PLOTS CLASS

# Arguments:
# filename = path to file with halo data
# first_x_field = field of x axis on the primary plot
# first_x_field = field of y axis on the primary plot
# first_x_field = fields of y axis options on the secondary plot

class linked_plots:
    
    def __init__(self, filename, first_x_field, first_y_field, second_y_fields):
        self.a = ytree.load(filename)
        self.complete_field_list = self.a.field_list + self.a.derived_field_list
        self.first_x_field = first_x_field
        self.first_y_field = first_y_field
        self.second_y_fields = second_y_fields
    
    @property
    def filename(self):
        return self._filename
    @property
    def first_x_field(self):
        return self._first_x_field
    @property
    def first_y_field(self):
        return self._first_y_field
    @property
    def second_y_fields(self):
        return self._second_y_fields
    
    @first_x_field.setter
    def first_x_field(self, x):
        if x not in self.complete_field_list:
            raise Exception("Invalid field: " + x)
        self._first_x_field = x
    @first_y_field.setter
    def first_y_field(self, y):
        if y not in self.complete_field_list:
            raise Exception("Invalid field: " + y)
        self._first_y_field = y
    @second_y_fields.setter
    def second_y_fields(self, ys):
        for f in ys:
            if f not in self.complete_field_list:
                raise Exception("Invalid field: " + f)
        self._second_y_fields = ys
            
    def show(self):
        # Creates redshift dictionary with self.fields
        redshift_dict = {}
        field_list = self.second_y_fields + [self.first_x_field] + [self.first_y_field]
        for tree in self.a:
            keys = np.round(tree["prog",'redshift'],2)
            for k,key in enumerate(keys):
                if key in redshift_dict:
                    for item in field_list:
                        redshift_dict[key][item] = np.append(redshift_dict[key][item],tree["prog",item][k])
                else:
                    redshift_dict[key] = {}
                    for item in field_list:
                        redshift_dict[key][item] = np.array(tree["prog",item][k])

        np.warnings.filterwarnings('ignore')

        sc_x = LinearScale()
        sc_y = LinearScale()
        sc_xax = Axis(label=(self.first_x_field), scale=sc_x, orientation='horizontal')
        sc_yax = Axis(label=(self.first_y_field), scale=sc_y, orientation='vertical')
        sc_x2 = LinearScale()
        sc_y2 = LinearScale()
        sc_xax2 = Axis(label=(self.first_x_field), scale=sc_x2, orientation='horizontal')
        sc_yax2 = Axis(label=(self.first_y_field), scale=sc_y2, orientation='vertical')

        Z = 0.00
        chosen_y_axis = redshift_dict[Z][self.second_y_fields[0]]
        x_y_cond = np.ones(len(chosen_y_axis),dtype=bool)

        x_orig = redshift_dict[Z][self.first_x_field]
        y_orig = redshift_dict[Z][self.first_y_field]

        scatter = bqplot.Scatter(x = x_orig, y = y_orig, 
                                scales = {'x': sc_x, 'y': sc_y}, colors = ['blue'])
        scatter2 = bqplot.Scatter(x = x_orig, y = chosen_y_axis,
                                scales = {'x': sc_x2, 'y': sc_y2}, colors = ['red'], visible = True)

        # Y-axis selector widget
        def dropdown_callback(change):
            highlight = bselect.selected
            Z = redshift_slider.value
            x_orig = redshift_dict[Z][self.first_x_field]
            y_orig = redshift_dict[Z][self.first_y_field]
            chosen_y_axis = redshift_dict[Z][change["new"]]
            try:
                x_condition = (x_orig>=float(highlight[0][0])) & (x_orig<=float(highlight[1][0]))
                y_condition = (y_orig>=float(highlight[0][1])) & (y_orig<=float(highlight[1][1]))
                x_y_cond = x_condition & y_condition
                scatter2.x = x_orig[x_y_cond]
                scatter2.y = chosen_y_axis[x_y_cond]
                fig_scat_brush2.axes =[Axis(label=(self.first_x_field), scale=sc_x2, orientation='horizontal'),
                           Axis(label=(change['new']), scale=sc_y2, orientation='vertical')]
                fig_scat_brush2.title = self.first_x_field + ' vs. ' + change['new']
                scatter2.visible = True
            except IndexError:
                ;

        y_selector = widgets.Dropdown(handler = dropdown_callback,options = self.second_y_fields,
                                      value = self.second_y_fields[0],description = 'Y Axis:')

        y_selector.observe(dropdown_callback, names=['value'])

        # Brush selector widget
        db_scat_brush = HTML(value='[]')

        def brush_callback(change):    
            highlight = bselect.selected
            Z = redshift_slider.value
            db_scat_brush.value = str(highlight)
            key = y_selector.value
            x_orig = redshift_dict[Z][self.first_x_field]
            y_orig = redshift_dict[Z][self.first_y_field]
            chosen_y_axis = redshift_dict[Z][key]
            if change.new is not None:
                try:
                    scatter2.visible = False
                    x_condition = (x_orig>=float(highlight[0][0])) & (x_orig<=float(highlight[1][0]))
                    y_condition = (y_orig>=float(highlight[0][1])) & (y_orig<=float(highlight[1][1]))
                    x_y_cond = x_condition & y_condition
                    scatter2.x = x_orig[x_y_cond]
                    scatter2.y = chosen_y_axis[x_y_cond]
                    fig_scat_brush2.axes =[Axis(label=(self.first_x_field), scale=sc_x2, orientation='horizontal'),
                               Axis(label=(key), scale=sc_y2, orientation='vertical')]
                    scatter2.visible = True
                except IndexError:
                    ;

        bselect = bqplot.interacts.BrushSelector(x_scale=sc_x, y_scale=sc_y, marks=[scatter],clear=False)

        bselect.observe(brush_callback, names=['selected'])

        fig_scat_brush1 = Figure(marks=[scatter], axes=[sc_xax, sc_yax], title='Vrms vs. Mvir',
                                interaction=bselect)
        fig_scat_brush2 = Figure(marks=[scatter2], axes=[sc_xax2, sc_yax2], title='Vrms vs. Mvir')

        # Redshift slider widget
        z_values = np.array(list(redshift_dict.keys()))
        redshift_slider = widgets.SelectionSlider(options=z_values,value=0.00,description='Redshift',
                                                  disabled=False,continuous_update=False,
                                                  orientation='horizontal',readout=True,
                                                  layout=Layout(width='100%',position='top'))

        def on_value_change(change):
            Z = change['new']
            try:
                scatter2.visible = False
                bselect.reset()
                x_orig = redshift_dict[Z][self.first_x_field]
                y_orig = redshift_dict[Z][self.first_y_field]
                scatter.x = x_orig
                scatter.y = y_orig
                fig_scat_brush1.axes =[Axis(label=(self.first_x_field), scale=sc_x, orientation='horizontal'),
                                       Axis(label=(self.first_y_field), scale=sc_y, orientation='vertical')]
                key = y_selector.value
                chosen_y_axis = redshift_dict[Z][key]
                scatter2.x = x_orig
                scatter2.y = chosen_y_axis
                fig_scat_brush2.axes =[Axis(label=(self.first_x_field), scale=sc_x2, orientation='horizontal'),
                                       Axis(label=(key), scale=sc_y2, orientation='vertical')]
                db_scat_brush.value = str(bselect.selected)
                scatter2.visible = True
            except TraitError:
                scatter2.visible = False
                bselect.reset()
                x_orig = [redshift_dict[Z][self.first_x_field]]
                y_orig = [redshift_dict[Z][self.first_y_field]]
                scatter.x = x_orig
                scatter.y = y_orig
                fig_scat_brush1.axes =[Axis(label=(self.first_x_field), scale=sc_x, orientation='horizontal'),
                                       Axis(label=(self.first_y_field), scale=sc_y, orientation='vertical')]
                key = y_selector.value
                chosen_y_axis = [redshift_dict[Z][key]]
                scatter2.x = x_orig
                scatter2.y = chosen_y_axis
                fig_scat_brush2.axes =[Axis(label=(self.first_x_field), scale=sc_x2, orientation='horizontal'),
                                       Axis(label=(key), scale=sc_y2, orientation='vertical')]
                db_scat_brush.value = "One source at z = "+str(Z)+" with (x,y) = ("+str(x_orig[0])+","+str(y_orig[0])+")"
                scatter2.visible = True

        redshift_slider.observe(on_value_change, names='value')

        left_box = VBox([db_scat_brush, fig_scat_brush1])
        right_box = VBox([fig_scat_brush2, y_selector])
        graph_box = HBox([left_box, right_box])

        return VBox([graph_box,redshift_slider])

In [125]:
linked_plot_widget = linked_plots("rockstar_halos/out_0.list", "Vrms", "Mvir", ["Rvir","Spin","velocity_z"])

In [126]:
linked_plot_widget.show()

VBox(children=(HBox(children=(VBox(children=(HTML(value='[]'), Figure(axes=[Axis(label='Vrms', scale=LinearScale()), Axis(label='Mvir', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=BrushSelector(marks=[Scatter(colors=['blue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  335.45999146,   335.36999512,  1145.06005859,   508.08999634,
           0.        ,   859.2199707 ,   466.19000244,   285.41000366,
         953.70001221,   968.47998047,   403.07998657,   564.36999512,
        1282.77001953,   353.1499939 ,   280.5       ,   227.82000732,
        1162.29003906,   441.38000488,   158.53999329,   390.5       ,
         764.47998047,  1156.84997559,   493.05999756,   222.91000366,
        1041.26000977,   603.98999023,   202.52999878,   311.6499939 ,
         365.3500061 ,   329.73001099,   296.38000488,   270.8500061 ,
         199.02000427,  1046.29003906,   521.42999268,   201.38000488,
         433.92999268,   885.86999512,   222.08000183,   977.70001221,
         293.55999756,   473.73999023,   682.72998047,   518.71002197,
         846.51000977,   565.73999023,   724.17999268,   873.57000732,
         239.41000366,   830.33001709,   178.53999329,   700.54998779,
         465.92001343,   660.29998779,   490.92001343,   765.36999512,
         402.07000732,   486.25      ,   494.54998779,   120.98000336,
         417.60998535,   408.04000854,   335.66000366,   515.15997314,
         684.85998535,   446.3999939 ,   445.20999146,   486.55999756,
         602.88000488,   247.32000732,   346.36999512,   224.3500061 ,
         245.44999695,   715.4699707 ,   677.        ,   519.57000732,
         515.09002686,   629.48999023,   126.88999939,   134.55999756,
         592.65997314,   824.5       ,   476.29000854,    69.91999817,
         468.92999268,   723.11999512,   671.53997803,   671.01000977,
         703.25      ,   631.15002441,   691.79998779,   632.76000977,
         300.98001099,   655.54998779,   315.8999939 ,   461.48999023,
         342.19000244,   630.0300293 ,   646.75      ,   362.58999634,
         457.76000977,   285.27999878,   549.66998291,   429.67999268,
         426.8999939 ,   361.70001221,   143.02999878,   450.61999512,
           0.        ,   641.17999268,   549.41998291,   166.91999817,
         564.07000732,   430.29000854,   550.53997803,   617.11999512,
         260.07000732,   604.88000488,   432.30999756,   464.30999756,
         130.63000488,   492.82998657,   603.94000244,   503.13000488,
         316.98999023,   578.02001953,   539.65002441,   199.83000183,
         570.78997803,   564.63000488,   535.7800293 ,   454.45001221,
         500.73001099,   590.45001221,   243.07000732,   503.76998901,
         468.61999512,   215.1000061 ,   438.29998779,   646.58001709,
         445.95001221,   321.48999023,   568.73999023,   441.76998901,
         558.28997803,   442.51998901,   366.57998657,   165.8999939 ,
         314.98999023,   484.47000122,   465.57998657,   297.5       ,
         529.65002441,   216.63999939,   448.64001465,   439.67001343,
         430.69000244,   376.3500061 ,   280.32000732,   491.10998535,
         485.45001221,   492.38000488,   330.75      ,   346.60998535,
         294.04998779,   422.48999023,   509.92001343,   486.33999634,
          58.38999939,   348.11999512,   455.01998901,   293.61999512,
         435.97000122,   327.73001099,   323.48001099,   334.07998657,
         478.77999878,   388.29000854,   422.19000244,   365.17999268,
         404.79998779,   362.70001221,   141.46000671,   367.52999878,
         465.6000